# Pokemon Data Analysis

Using [PokéAPI](https://pokeapi.co/), I will create a data pipeline in python to extract and transform data about Pokémon in a MySQL database. From there, I’ll analyse the different aspects of the data in python and SQL

In [1]:
import pokemon_pipline as pk
import pandas as pd
from sqlalchemy import create_engine

## Data Pipeline

### 1. Extract and Transform Data

Firstly, by using the requests package and a Pokémon object, I will create a list containing the information of every Pokémon, then convert the list into a pandas DataFrame as a more easily accessible form.

In [2]:
pokemon_base_url: str = "https://pokeapi.co/api/v2/pokemon/"
pokemon_list: list = pk.import_pokemon_data_to_list(25,url = pokemon_base_url)

pokemon_df: pd.DataFrame = pk.list_pokemon_data_to_pd(pokemon_list)
pokemon_df.head()

,pokemonId,name,type1,type2,height,weight,hp,attack,defense,special_attack,special_defense,speed
0,1,bulbasaur,grass,poison,7,69,45,49,49,65,65,45
1,2,ivysaur,grass,poison,10,130,60,62,63,80,80,60
2,3,venusaur,grass,poison,20,1000,80,82,83,100,100,80
3,4,charmander,fire,None,6,85,39,52,43,60,50,65
4,5,charmeleon,fire,None,11,190,58,64,58,80,65,80


I will do the same for Pokémon types, ending up with a DataFrame of Pokémon types and a corresponding ID


In [3]:
type_base_url: str = "https://pokeapi.co/api/v2/type/"
pokemon_types: list = pk.import_type_data_to_list(url = type_base_url)

types_df: pd.DataFrame = pk.list_types_data_to_pd(pokemon_types)
types_df.head()

,pokemonType,typeID
0,normal,0
1,fighting,1
2,flying,2
3,poison,3
4,ground,4


### 2. Load Data to Repository

Can load the Pokémon data in a csv and parquet file

In [ ]:
pk.data_pd_to_csv(pokemon_df)
pk.data_pd_to_parquet(pokemon_df)

However, for more functionality, I will export both DataFrames into a MySQL Database. The following Database and tables are created using mysql.connector. As a Pokémon can have more than one type, the Database is in 2nd normal form to remove duplicate data rows and ensure the data dependencies.       
![Pokémon Database erd](PokemonDB_erd.png)

In [4]:
pk.create_db()
pk.create_db_tables()

Can now populate the database with mysql.connector, sqlalchemy and pandas, making sure to insert the table data in the right order.    
This will only work if all the tables are empty, otherwise the data insertion will fail


In [5]:
#if the tables are not empty
#pk.reset_db_table("all")

#insert pokemon data
pk.insert_pokemon_data_to_db(pokemon_df)

#insert type data
pk.insert_types_data_to_db(types_df)

# insert many to many table data
pk.insert_many_many_data_to_db(pokemon_df,types_df)

## Data Analysis

Now

### Data from sql to python (WIP)

With the data in the database, I can now query the data using sqlalchemy and pandas

In [6]:
engine = create_engine(pk.SQL_ALCHEMY_CON_STRING)

 -------   query the whole database

In [9]:
whole_db_str: str = \
"""
    SELECT *
    FROM pokemon p
    inner Join pokemontypes pt
    on p.pokemonId = pt.pokemonId
    join types t
    on t.typeId = pt.typeId
    order by p.pokemonId;
"""

In [13]:
df_whole_db =pd.read_sql(whole_db_str, engine)
df_whole_db.head()

,pokemonId,name,height,weight,hp,attack,defense,special_attack,special_defense,speed,pokemonId,typeId,typeId,pokemonType
0,1,bulbasaur,7.0,69.0,45,49,49,65,65,45,1,3,3,poison
1,1,bulbasaur,7.0,69.0,45,49,49,65,65,45,1,11,11,grass
2,2,ivysaur,10.0,130.0,60,62,63,80,80,60,2,3,3,poison
3,2,ivysaur,10.0,130.0,60,62,63,80,80,60,2,11,11,grass
4,3,venusaur,20.0,1000.0,80,82,83,100,100,80,3,3,3,poison


get data from sql in each query or just use pandas

for each question:  
type   
height/weight    
stats